In [1]:
import sys
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

sys.path.append('..')
from audio_toolbox.metrics import splitTabularPredicting

audio_root_folder = '../archive/Data'
labels_csv = os.path.join(audio_root_folder, 'dataIndex.csv')
df = pd.read_csv(labels_csv, header=0)
copy = pd.read_csv(labels_csv, header=0)
slicedCSV = os.path.join(audio_root_folder, 'feature.csv')
slicedDataSet = pd.read_csv(slicedCSV, header=0)
label = df['label']
df = df.sample(frac=1, random_state=42).reset_index(drop=False)
slicedDataSet = slicedDataSet.sample(frac=1, random_state=42).reset_index(drop=False)

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(slicedDataSet, train_size = 0.1)
X_train_drop = X_train.drop(['label','index'], axis=1, inplace=False)

In [3]:
tree = RandomForestClassifier(max_depth = 15, max_features = 5)
tree.fit(X_train_drop, X_train['label'])

RandomForestClassifier(max_depth=15, max_features=5)

In [4]:
from audio_toolbox.metrics import splitTabularPredicting
_, X_test = train_test_split(df, test_size= 0.5)
prediction= splitTabularPredicting(tree, X_test, slicedDataSet)

In [5]:
from sklearn.metrics import accuracy_score

# Assuming `prediction` is your list of predictions
accuracy = accuracy_score(X_test['label'], prediction)
print(f"Accuracy: {accuracy}")

Accuracy: 0.932


In [39]:
class splitForest:
    def __init__(self):
        self.tree = RandomForestClassifier(max_depth = 15, max_features = 5)
        self.dataset = slicedDataSet
    def fit(self, X_train, Y_train):
        self.tree.fit(X_train, Y_train)
    def predict(self, X_test):
        return splitTabularPredicting(self.tree, X_test, self.dataset)

array(['metal', 'jazz', 'reggae', 'classical', 'country', 'rock',
       'reggae', 'rock', 'country', 'rock', 'classical', 'hiphop',
       'hiphop', 'disco', 'hiphop', 'hiphop', 'metal', 'reggae', 'metal',
       'country', 'hiphop', 'country', 'metal', 'hiphop', 'metal',
       'blues', 'rock', 'reggae', 'hiphop', 'disco', 'hiphop', 'pop',
       'pop', 'country', 'pop', 'disco', 'hiphop', 'rock', 'blues',
       'metal', 'hiphop', 'country', 'pop', 'jazz', 'metal', 'blues',
       'metal', 'classical', 'country', 'reggae', 'reggae', 'country',
       'rock', 'jazz', 'jazz', 'metal', 'jazz', 'classical', 'classical',
       'blues', 'reggae', 'classical', 'rock', 'reggae', 'metal', 'jazz',
       'classical', 'hiphop', 'reggae', 'metal', 'pop', 'classical',
       'pop', 'rock', 'blues', 'hiphop', 'hiphop', 'disco', 'jazz',
       'jazz', 'jazz', 'jazz', 'country', 'reggae', 'jazz', 'disco',
       'disco', 'hiphop', 'rock', 'hiphop', 'country', 'blues', 'reggae',
       'blues', 'hi

In [40]:
X_test['label']

402      metal
818       jazz
344     reggae
929       jazz
54       blues
        ...   
315    country
785       rock
930      disco
884    country
988     hiphop
Name: label, Length: 500, dtype: object

In [2]:
data_folds = []
target_folds = []
K = 5
tree = RandomForestClassifier(max_depth = 15, max_features = 5)
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
fold_size = len(df) // K

df.drop(['label'], axis=1, inplace=True)
df.columns = df.iloc[0]
for i in range(5):
    start = i * fold_size
    end = start + fold_size if i < K-1 else len(df)  # Ensure the last fold includes the remainder
    data_folds.append(df[start:end])
    target_folds.append(label[start:end])

In [3]:
accuracies = []

for i in range(K):
    X_train = pd.concat([data_folds[j] for j in range(K) if j != i], ignore_index=True)
    y_train = pd.concat([target_folds[j] for j in range(K) if j != i], ignore_index=True)
    X_test = data_folds[i]
    y_test = target_folds[i]
    tree.fit(X_train, y_train)
    prediction= splitTabularPredicting(tree, X_test, slicedDataSet)
    accuracy = accuracy_score(y_test, prediction)
    accuracies.append(accuracy)
    
average_accuracy = sum(accuracies) / len(accuracies)
print(f"Average Accuracy: {average_accuracy}")

KeyError: 'index'